In [8]:
import torch
import os
import torchvision
from taco_dataset import TACODataset
from model import get_model
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torchmetrics.detection.mean_ap import MeanAveragePrecision
import numpy as np
import pandas as pd


In [3]:
def evaluate_model(model, data_loader, device):
    """
    Evaluate the model on the given data loader
    Args:
        model: the model to evaluate
        data_loader: the data loader to evaluate the model on
        device: the device to evaluate the model on (cuda / cpu)
    Returns:
        metric: the metrics to evaluate the model (mAP, precision, recall, etc.)
    """
    model.eval()
    metric = MeanAveragePrecision()
    
    with torch.no_grad():
        for images, targets in data_loader:
            images = list(img.to(device) for img in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            # predict and metric update
            predictions = model(images)
            metric.update(predictions, targets)

    result = metric.compute()
    return result


In [4]:
def visualize_predictions(model, dataset, idx, device, confidence_threshold=0.5):
    """
    Visualize the predictions of the model on the given dataset
    Args:
        model: the model to evaluate
        dataset: the dataset to evaluate the model on
        idx: the index of the image to visualize
        device: the device to evaluate the model on (cuda / cpu)
        confidence_threshold: the confidence threshold to visualize the predictions
    """
    image, target = dataset[idx]

    # predict
    model.eval()
    with torch.no_grad():
        prediction = model([image.to(device)])
        prediction = prediction[0]

    # original image
    image = torch.tensor(image)
    image = torchvision.transforms.functional.normalize(
        image,
        mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
        std=[1/0.229, 1/0.224, 1/0.225]
    )
    image = image.permute(1, 2, 0).cpu().numpy()

    # visualize the results
    plt.figure(figsize=(12, 8))
    plt.imshow(image)
    # plt.title("Ground Truth")
    # plt.axis('off')
    # plt.show()

    # visualize the ground truth boxes (green)
    for box, label in zip(target['boxes'], target['labels']):
        box = box.cpu().numpy()
        rect = patches.Rectangle(
            (box[0], box[1]),
            box[2] - box[0],
            box[3] - box[1],
            linewidth=2,
            edgecolor='g',
            facecolor='none'
        )
        plt.gca().add_patch(rect)
        plt.text(
            box[0], box[1]-5,
            f'GT: {dataset.cat_ids[label.item()]}',
            color='white', bbox=dict(facecolor='green', alpha=0.5)
        )
    
    # 예측 박스 (빨간색)
    for box, label, score in zip(prediction['boxes'], prediction['labels'], prediction['scores']):
        if score > confidence_threshold:
            box = box.cpu().numpy()
            rect = patches.Rectangle(
                (box[0], box[1]), box[2]-box[0], box[3]-box[1],
                linewidth=2, edgecolor='r', facecolor='none'
            )
            plt.gca().add_patch(rect)
            plt.text(
                box[0], box[1]-20,
                f'Pred: {dataset.cat_ids[label.item()]}: {score:.2f}',
                color='white', bbox=dict(facecolor='red', alpha=0.5)
            )
    
    plt.axis('off')
    plt.show()

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_dir = './data'
checkpoints_dir = './checkpoints'

# load test dataset
test_dataset = TACODataset(root_dir=data_dir, annotation_file='annotations_0_test.json')
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0, collate_fn=lambda x: tuple(zip(*x)))

# 모든 체크포인트 파일 찾기
checkpoint_files = [f for f in os.listdir(checkpoints_dir) if f.startswith('model_epoch_') and f.endswith('.pth')]
checkpoint_files.sort(key=lambda x: int(x.split('_')[2].split('.')[0]))  # epoch 번호순으로 정렬

# 결과를 저장할 딕셔너리
results_dict = {
    'epoch': [],
    'mAP': [],
    'mAP_50': [],
    'mAP_75': []
}

# 각 체크포인트별로 평가 수행
for checkpoint_file in checkpoint_files:
    epoch = int(checkpoint_file.split('_')[2].split('.')[0])
    checkpoint_path = os.path.join(checkpoints_dir, checkpoint_file)
    print(f"\nEvaluating checkpoint: {checkpoint_file}")
    
    # 모델 로드
    model = get_model(test_dataset.num_classes)
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    
    # 평가 수행
    results = evaluate_model(model, test_loader, device)
    
    # 결과 저장
    results_dict['epoch'].append(epoch)
    results_dict['mAP'].append(results['map'].item())
    results_dict['mAP_50'].append(results['map_50'].item())
    results_dict['mAP_75'].append(results['map_75'].item())
    
    print(f"Epoch {epoch}:")
    print(f"mAP @ 0.5:0.95: {results['map'].item():.4f}")
    print(f"mAP @ 0.5: {results['map_50'].item():.4f}")
    print(f"mAP @ 0.75: {results['map_75'].item():.4f}")

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results_dict)
print("\nAll Results:")
print(results_df)

# 결과 시각화
plt.figure(figsize=(10, 6))
plt.plot(results_df['epoch'], results_df['mAP'], label='mAP@0.5:0.95', marker='o')
plt.plot(results_df['epoch'], results_df['mAP_50'], label='mAP@0.5', marker='o')
plt.plot(results_df['epoch'], results_df['mAP_75'], label='mAP@0.75', marker='o')
plt.xlabel('Epoch')
plt.ylabel('mAP')
plt.title('Model Performance across Epochs')
plt.legend()
plt.grid(True)
plt.show()

# 최고 성능 체크포인트 찾기
best_map = results_df.loc[results_df['mAP'].idxmax()]
print("\nBest Checkpoint:")
print(f"Epoch: {best_map['epoch']}")
print(f"mAP @ 0.5:0.95: {best_map['mAP']:.4f}")
print(f"mAP @ 0.5: {best_map['mAP_50']:.4f}")
print(f"mAP @ 0.75: {best_map['mAP_75']:.4f}")

# 최고 성능 모델로 예시 이미지 시각화
best_checkpoint_path = os.path.join(checkpoints_dir, f"model_epoch_{int(best_map['epoch'])}.pth")
best_model = get_model(test_dataset.num_classes)
best_model.load_state_dict(torch.load(best_checkpoint_path)['model_state_dict'])
best_model.to(device)

print("\nVisualizing predictions with best model:")
num_visualizations = 5
for i in range(num_visualizations):
    print(f"\nExample {i+1}/{num_visualizations}")
    visualize_predictions(best_model, test_dataset, i, device)


FileNotFoundError: [Errno 2] No such file or directory: 'annotations_0_test.json'